# Projeto 1 - Ciência dos Dados

Nome: Luca Coutinho Melão

Nome: Thiago de Donato Youssef

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [63]:
!pip install emoji

In [64]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [65]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [72]:
# Base de dados para treinamento
train = pd.read_excel(filename)

# Base de dados para teste
test = pd.read_excel(filename, sheet_name = 'Teste')

___
## Classificador automático de sentimento


### Descrição do produto e proposta
O Playstation 5 (PS5) é o novo console de jogos da Sony, com seu lançamento mundial oficial em 12 de novembro de 2020. O console é  pioneiro na nova geração de games junto de seu concorrente da Microsoft, o Xbox Series X. Anunciado oficialmente em 11 de junho de 2020, o PS5 já obteve impressionantes números de pré-venda desde a divulgação de seu preço e jogos exclusivos em 16/09/2020, e promete apresentar grandes diferenciais e avanços em relação ao seu antecessor, o PS4.

Nossa proposta é a elaboração de um classificador Naive Bayes que categorize tweets de usuários sobre o PS5 como relevantes ou irrelevantes, levando em conta o conteúdo dos mesmos de acordo com os critério mencionados abaixo:

#### Critérios adotados para relevância:

Tweets que expressam o sentimento do usuário com relação ao console, remetem geralmente às expectativas sobre os preços, jogos mais aguardados, comparações com a concorrência.

#### Critérios adotados para irrelevância:

Tweets que não estão relacionados ao produto, ou que não agregam nada à análise de sentimentos sobre ele. Tais como: anúncios de lives, divulgação de contúedos não relacionados ao produto e comentários sem significado algum.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [68]:
# Separa emojis dos outros caracteres
def separa_emoji(text):
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

# Fonte : https://medium.com/analytics-vidhya/simplify-your-dataset-cleaning-with-pandas-75951b23568e

def clean_df(dados,base):
    """
        Função que limpa caracteres especiais e troca por espaços.
    """
    spec_chars = ["!","˜","rt","\n",'"',"#","%","&","'","(",")",
                  "*","+",",","-",".","/",":",";","<",
                  "=",">","?","[","\\","]","^","_",
                  "`","{","|","}","~","–","https","@"]
    for char in spec_chars:
        dados[base] = dados[base].str.replace(char, ' ')

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços.
    """
    #import string
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

# Dicionários que guardam quantas vezes uma palavra apareceu na base de dados de acordo com a sua relevância
dc_0 = dict()
dc_1 = dict()

a = 1
b = 1
    
# Treinamento para classificação de tweets irrelevantes
for tweet in train.loc[train.Relevância==0, 'Treinamento']:
    tweet = cleanup(tweet)
    words = tweet.split()
    for word in words:
        if word not in dc_0.keys():
            dc_0[word] = 1
        else:
            dc_0[word] += 1
            
# Treinamento para classificação de tweets relevantes
for tweet in train.loc[train.Relevância==1, 'Treinamento']:
    tweet = cleanup(tweet)
    words = tweet.split()
    for word in words:
        if word not in dc_1.keys():
            dc_1[word] = 1
        else:
            dc_1[word] += 1
    
for i in dc_0.values():
    a += i
for i in dc_1.values():
    b += i
        
den = len(dc_1)
repetido = False
    
for i in dc_0.keys():
    for x in dc_1.keys():
        if i == x:
            repetido = True
    if repetido == False:
        den += 1
            
den_0 = a+den
den_1 = b+den
            
dic = [dc_0,dc_1,den_0,den_1]

# Função que calcula a probabilidade de um tweet não ser relevante
def calcula_prob_0(tweet,dic):
    tweet = cleanup(tweet)
    dc_0 = dic[0]
    den = dic[2]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_0.keys():
            prob *= ((dc_0[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que calcula a probabilidade de um tweet ser relevante
def calcula_prob_1(tweet,dic):
    tweet = cleanup(tweet)
    dc_1 = dic[1]
    den = dic[3]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_1.keys():
            prob *= ((dc_1[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que classifica os tweets
def classificador(dados,base,dic):
    clean_df(dados,base)
    for tweet in dados[base]:
        P_T_dado_R = calcula_prob_1(tweet,dic)
        P_T_dado_Rc = calcula_prob_0(tweet,dic)
        if (P_T_dado_R > P_T_dado_Rc):
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = 1
        else:
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = 0
        separa_emoji(tweet)
        


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [71]:
# Classificando base de testes
classificador(test,'Teste',dic)

# Tabela que demonstra o teste do classificador com a base de Testes
tab = pd.crosstab(test.Relevância, test.Relevância_Teste)
print('Tabela cruzada: classifiação manual X classificação por Naive Bayes')
print(tab)

# Porcentagens de erros/acertos do classificador Naive Bayes

VP = tab[1][1]
FP = tab[1][0]
VN = tab[0][0]
FN = tab[0][1]
total_positivo = VP + FP 
total_negativo = VN + FN
total = VP + FP + VN + FN

print('-----------------------------------------------')
print(f'Porcentagem de verdadeiros positivos: {(VP/total_positivo)*100:.2f}%')
print(f'Porcentagem de falsos positivos: {(FP/total_positivo)*100:.2f}%')
print(f'Porcentagem de verdadeiros negativos: {(VN/total_negativo)*100:.2f}%')
print(f'Porcentagem de falsos negativos: {(FN/total_negativo)*100:.2f}%')
print('-----------------------------------------------')
print(f'Porcentagem de acerto geral: {((VP + VN)/total)*100:.2f}%')


Tabela cruzada: classifiação manual X classificação por Naive Bayes
Relevância_Teste  0.0  1.0
Relevância                
0                  99   69
1                  38  147
-----------------------------------------------
Porcentagem de verdadeiros positivos: 68.06%
Porcentagem de falsos positivos: 31.94%
Porcentagem de verdadeiros negativos: 72.26%
Porcentagem de falsos negativos: 27.74%
-----------------------------------------------
Porcentagem de acerto geral: 69.69%


___
### Conclusão

#### Comparativo qualitativo sobre os percentuais obtidos

Tendo em vista as porcentagens de erros/acertos de nosso classificador, pode-se dizer que sua performance mostrou-se parcialmente satifatória. No caso de tweets relevantes, ele demonstrou uma taxa de acerto na classificação de 68.06%, enquanto no caso de tweets irrelevantes, ele apresentou uma taxa de acerto superior, de 72.26%.

Em nossa visão, um desempenho satisfatório deveria ser de 80% de acerto para cima. Por isso, considerando o treinamento com uma única base de dados, consideramamos o resultado final obtido tolerável.

A partir das porcentagens, pode-se concluir que o classificador possui um melhor desempenho na classificação de tweets como irrelevantes.


#### Como são tratadas as mensagens com dupla negação e sarcasmo?

O tratamento de mensagens com dupla negação e sarcasmo não se faz necessário no caso de nosso classificador, uma vez que de acordo com os critérios de classificação estabelecidos, mensagens que trazem opiniões/sentimentos tanto positivos quanto negativos acerca do produto em questão são consideradas relevantes, sendo categorizadas igualmente. 

#### Plano de expansão. Por que nosso projeto deve continuar sendo financiado?
Nosso projeto, em sua primeira versão, obteve uma taxa de acerto geral de 69.69%. Para isso, foi levado em conta apenas uma única base de dados para treinamento, e uma outra única base para teste. Sabemos que, com o tempo, é possível aperfeiçoar nosso classificador Naive Bayes através dos seguintes práticas:
- Uso de novas bases de dados diversificadas para treinamento. 
- Filtragem de palavras que não adicionam significado ou conteúdo ao texto, tais como conectivos, artigos de ligação, etc.
- Categorização de palavras em expressões pelo seu emprego usual nas frases. 
- Agrupamento de palavras de acordo com seu campo semântico.

Portanto, adotando tais técnicas, conseguiremos melhorar o desempenho de nosso classificador, e por isso nosso projeto deve continuar sendo financiado.

### Por que não posso usar o próprio classificador para gerar mais amostras de treinamento?
Ao utilizar o próprio classificador para gerar mais amostras de treinamento, os eventuais erros cometidos por ele serão armazenados e propagados adiante. Dessa forma, novas amostras de treinamento geradas por ele mesmo carregariam erros consigo e diminuiriam a eficácia do classificador.

## Diferentes cenários para Naive Bayes fora do contexto do projeto

### Primeiro cenário: Caixa de spam do email
A partir de uma análise de palavras-chave e assuntos recorrentes, o classificador Naive Bayes é capaz de categorizar emails que contém promoções, ofertas, anúncios, etc, como "spam".

### Segundo cenário: Correntes de fake news em redes sociais
As correntes de redes sociais que contém fake news usualmente seguem uma tendência, elas apresentam textos em caps-lock, não apresentam fontes/links confiáveis, apelam para o discurso emotivo, e pedem para que sejam imediatamente compartilhadas. A partir da identificação de tais elementos no texto, o classificador Naive Bayes é capaz de sinalizar quando a mensagem contém fake news, ou pelo menos apontar quando há alta probabilidade disso.

### Terceiro cenário: Algoritmos de recomendação
Em aplicativos como Netflix ou Spotify é comum ver uma seção de "recomendados para você" ou "acho que você vai gostar". Tendo o conteúdo consumido pelo usuário em um período recente como base de treinamento, o classificador Naive Bayes pode ser utilizado para recomendar novos títulos que possivelmente irão agradar o usuário. Para isso, ele levará em conta o gênero do filme/música, ano de lançamento, palavras das "lyrics", etc.

### Quarto cenário: Classificação de documentos
No meio corporativo e jurídico é muito comum a necessidade de lidar com diversos documentos, tais como procurações, petições, contratos sociais, acordos de confidencialidade, etc. Sendo assim, a partir de palavras-chave presentes em cada tipo, o classificador Naive Bayes é capaz de categorizar os documentos em seus respectivos grupos.

### Quinto cenário: Categorização de artigos 
Ao acessar uma newsletter ou ler as notícias, é muito comum que os artigos estejam divididos de acordo com suas categorias como esportes, tecnologia, economia, entre outros. O classificador Naive Bayes pode fazer essa divisão também levando em conta palavras-chave presentes em cada categoria.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**